In [1]:
import geopandas as gpd
import fiona
import os
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import configparser
import momepy
import libpysal
import osmnx
import pandas
from bokeh.io import output_notebook
from bokeh.plotting import show
from clustergram import Clustergram
from shapely.geometry import Point
local_crs = "EPSG:2039"
output_notebook()

c:\Users\david\Desktop\DSSG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading BokehJS ...

Load all the gdb files from michael folders

In [6]:

config = configparser.ConfigParser()
config_file = 'config.ini'
# check if file exists
if not os.path.isfile(config_file):
    # create a simple config file
    config['Paths'] = {'root_folder': ''}
    with open(config_file, 'w') as configfile:
        config.write(configfile)
    print(f"Created configuration file '{config_file}'.")

config.read('config.ini')

root_folder = config['Paths']['root_folder']  
print(root_folder)
root_folder = os.path.join(root_folder, "Michaels_Data")
print(root_folder)
# List all .gdb directories in the folder
gdb_files = []
for dirpath, dirnames, filenames in os.walk(root_folder):
    # Check if the folder contains a "commondata" folder
    if "commondata" in dirnames:
        commondata_folder = os.path.join(dirpath, "commondata")
        commondata_folder = os.path.normpath(commondata_folder)

        # List all .gdb files in the "commondata" folder
        for f in os.listdir(commondata_folder):
            if f.endswith('.gdb'):
                gdb_files.append(os.path.join(commondata_folder, f))

# Print the collected .gdb files
for gdb in gdb_files:
    print(gdb)
print("GDB Files:", gdb_files)

C:\\Users\\david\\Desktop
C:\\Users\\david\\Desktop\Michaels_Data
GDB Files: []


Calculating metrics over buildings:

In [3]:
# load municipal buildings
gdb_file = gdb_files[4]  # Modify if you want to choose a different .gdb
print(gdb_file)

# List the layers in the selected .gdb
layers = fiona.listlayers(gdb_file)
print("Layers in the selected GDB:", layers)

# Choose a specific layer within the .gdb
textures_layer = layers[0]  # Modify if you want to choose a different layer

# Load the specific layer
gdf = gpd.read_file(gdb_file, layer=textures_layer)

IndexError: list index out of range

In [84]:
buildings = gdf.to_crs(local_crs)

# remove the duplicates
duplicates = buildings[buildings.duplicated(subset='geometry', keep=False)]
buildings = buildings.drop_duplicates(subset='geometry', keep='first')
buildings = buildings.reset_index(drop=True)

In [56]:
# generates a random height for each building 
import numpy as np

# Generate random positive numbers
random_numbers = np.random.rand(len(buildings))

# Create a new column in the buildings DataFrame
buildings['building_height'] = random_numbers

In [58]:
# Calculate Building Height
# We need to have a 'height_col' or a similar column to calculate this

# Calculate Volume, requires floor_area and height
# Assuming height and area have been calculated and are stored in 'height' and 'floor_area' columns
buildings['floor_area'] = momepy.floor_area(buildings['Shape_Area'],buildings['building_height'])
buildings['volume'] = momepy.volume(buildings['Shape_Area'], buildings['building_height'])
buildings['form_factor'] = momepy.form_factor(buildings, buildings['building_height'])

In [64]:
# Basic geometric properties
buildings['perimeter'] = buildings.geometry.length
buildings['shape_index'] = momepy.shape_index(buildings,momepy.longest_axis_length(buildings))
buildings['circular_compactness'] = momepy.circular_compactness(buildings)
buildings['square_compactness'] = momepy.square_compactness(buildings)
buildings['weighted_axis_compactness'] = momepy.compactness_weighted_axis(buildings)
buildings['convexity'] = momepy.convexity(buildings)
buildings['courtyard_area'] = momepy.courtyard_area(buildings)
buildings['courtyard_index'] = momepy.courtyard_index(buildings)
buildings['corners'] = momepy.corners(buildings, include_interiors=True)   # include_interiors=False works only for polygons not multipolygons
buildings['fractal_dimension'] = momepy.fractal_dimension(buildings)
buildings['facade_ratio'] = momepy.facade_ratio(buildings)

In [65]:
# More complex morphological metrics
buildings['orientation'] = momepy.orientation(buildings)
buildings['longest_axis_length'] = momepy.longest_axis_length(buildings)
buildings['equivalent_rectangular_index'] = momepy.equivalent_rectangular_index(buildings)
buildings['elongation'] = momepy.elongation(buildings)
buildings['linearity'] = momepy.linearity(buildings)
buildings['rectangularity'] = momepy.rectangularity(buildings)
buildings['squareness'] = momepy.squareness(buildings, include_interiors=True)  
buildings['shared_walls_length'] = momepy.shared_walls(buildings)

In [66]:
# Metrics related to building adjacency and Graph
from libpysal import graph
delaunay = graph.Graph.build_triangulation(buildings.centroid,coplanar='clique').assign_self_weight()
orientation = momepy.orientation(buildings)
buildings['alignment'] = momepy.alignment(orientation, delaunay)

knn15 = graph.Graph.build_knn(buildings.centroid, k=15, coplanar='clique')  # adjust k if needed   
contiguity = graph.Graph.build_contiguity(buildings)
buildings['adjacency'] = momepy.building_adjacency(contiguity,knn15)
buildings['mean_interbuilding_distance'] = momepy.mean_interbuilding_distance(buildings, delaunay, knn15)
buildings['neighbour_distance'] = momepy.neighbor_distance(buildings, delaunay)
buildings['courtyards_num'] = momepy.courtyards(buildings, contiguity) # Calculate the number of courtyards within the joined structure

C:\Users\ameer\PycharmProjects\DSSG\.venv\Lib\site-packages\momepy\functional\_distribution.py:285: UserWarning: The indices of the two GeoSeries are different.
  (geoms.distance(geometry.geometry, align=True)).groupby(level=0).mean()


In [67]:
# the following metrics caculate the diversity index of building types, it's possible to override the binning method (refer to the full documentation)
from numbers import Number
import inequality  # required for Theil index
knn5 = graph.Graph.build_knn(buildings.centroid, k=5)
for col in buildings.columns:
    if isinstance(buildings[col],Number):
        buildings[f'{col}_shannon'] = momepy.shannon(buildings[col], knn5)
        buildings[f'{col}_simpson'] = momepy.simpson(buildings[col], knn5)
        buildings[f'{col}_theil'] = momepy.theil(buildings[col], knn5)
        buildings[f'{col}_values_range'] = momepy.values_range(buildings[col], knn5)
        buildings[f'{col}_mean_deviation'] = momepy.mean_deviation(buildings[col], knn5)

In [68]:
# This is a modified version of the momepy.buffered_limit function that allows for adaptive buffer calculation on our data
from libpysal import graph
from packaging.version import Version
import numpy as np
def buffered_limit(gdf,buffer: float | str = 100,min_buffer: float = 0,max_buffer: float = 100,**kwargs,):
   
    if buffer == "adaptive":
        
        gabriel = graph.Graph.build_triangulation(gdf.centroid, "gabriel", kernel="identity", coplanar='clique')
        max_dist = gabriel.aggregate("max")
        buffer = np.clip(max_dist / 2 + max_dist * 0.1, min_buffer, max_buffer).values

    elif not isinstance(buffer, int | float):
        raise ValueError("`buffer` must be either 'adaptive' or a number.")

    GPD_GE_10 = Version(gpd.__version__) >= Version("1.0dev")
    return (
        gdf.buffer(buffer, **kwargs).union_all()
        if GPD_GE_10
        else gdf.buffer(buffer, **kwargs).unary_union
    )

In [69]:
# metrics related to tessellation
limit = buffered_limit(buildings, buffer = 'adaptive')
tessellation = momepy.morphological_tessellation(buildings) # need adjustment
blg_orient = momepy.orientation(buildings)
tess_orient = momepy.orientation(tessellation)
buildings['cell_orientation'] = momepy.cell_alignment(blg_orient, tess_orient)
buildings['num_of_neighbours'] = momepy.neighbors(tessellation, contiguity, weighted=True)

Calculate metrics over streets

In [ ]:
place = "Jerusalem, Israel"
local_crs = "EPSG:2039"

# Geocode using Nominatim
# latitude = 31.7683
# longitude = 35.2137
# point = gpd.GeoDataFrame(geometry=[Point(longitude, latitude)], crs="EPSG:4326")

#Load open street buildings
osm_buildings = osmnx.features_from_place(place, tags={"building": True})
osm_buildings = osm_buildings[osm_buildings.geom_type == "Polygon"].reset_index(drop=True)
osm_buildings = osm_buildings[["geometry"]].to_crs(local_crs)
osm_buildings.head()
osm_graph = osmnx.graph_from_place(place, network_type="drive")
osm_graph = osmnx.projection.project_graph(osm_graph, to_crs=local_crs)
streets = osmnx.graph_to_gdfs(
    osmnx.convert.to_undirected(osm_graph),
    nodes=False,
    edges=True,
    node_geometry=False,
    fill_edge_geometry=True,
).reset_index(drop=True)
print(streets.columns)
streets_geometry = streets['geometry']

In [ ]:
streets['orientation'] = momepy.orientation(streets_geometry)
streets['longest_axis_length'] = momepy.longest_axis_length(streets_geometry)
# streets['perimeter_wall'] = momepy.perimeter_wall(streets_geometry)
streets['compactness_weighted_axis'] = momepy.compactness_weighted_axis(streets_geometry)
streets['linearity'] = momepy.linearity(streets_geometry)

streets['squareness'] = momepy.elongation(streets_geometry)
streets['alignment'] = momepy.alignment(streets['orientation'], osm_graph)
streets['cell_alignment'] = momepy.cell_alignment(streets['orientation'], osm_graph)
streets['neighbor_distance'] = momepy.neighbor_distance(streets_geometry, osm_graph)
streets['neighbors'] = momepy.neighbors(streets_geometry, osm_graph)
streets['betweenness_centrality'] = momepy.betweenness_centrality(osm_graph)



streets = momepy.remove_false_nodes(streets)
streets["length"] = streets.length
streets["linearity"] = momepy.linearity(streets)


streets = momepy.remove_false_nodes(streets)
streets["length"] = streets.length
streets["linearity"] = momepy.linearity(streets)

metrics that not included on streets: Non related metrics for streets: courtyard_area, perimeter_wall, centroid_corner_distance, circular_compactness (works on polygons), convexity, corners, courtyard_index(relevant for closed polygons), elongation(how streched the polygon is), equivalent_rectangular_index, facade ratio, form_factor, fractal_dimension, rectangularity, shape_index, squere compactness, 

In [ ]:
# shared metrics between buildings and streets
# Ensure you have the necessary context (like street networks) for these:
# gdf_network =   # the street network
# buildings['node_density'] = momepy.node_density(gdf_network, buildings, radius=300)
# buildings['street_profile'] = momepy.street_profile(gdf_network, buildings, radius=300)
# buildings['reach'] = momepy.Reach(buildings, gdf_network, distance=500)

blg_orient = momepy.orientation(buildings)
str_orient = momepy.orientation(streets)
momepy.street_alignment(blg_orient, str_orient, buildings["street_index"])
